In [7]:
#https://github.com/vi3k6i5/GuidedLDA
cd GuidedLDA
sh build_dist.sh
python setup.py sdist
pip install -e .


SyntaxError: invalid syntax. Perhaps you forgot a comma? (788299312.py, line 2)

In [ ]:
#pip install gensim nltk spacy


In [39]:
import spacy
import es_core_news_sm
import gensim
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
import unidecode
import re

#for bigrams
from gensim.models import Phrases


In [25]:
# ! python -m spacy download es_core_news_sm

### Reading the data

In [31]:
data = pd.read_csv('C:/Users/carme/Dropbox/MIO/DOCS/Hertie/Thesis/Hertie-Thesis/Data/clean_data.csv')
# 'C:/Users/carme/Dropbox/MIO/DOCS/Hertie/Thesis/Elecciones_Municipales_2024/clean_data.csv'
#filter column with na values
plans = data.copy().dropna()
plans = plans.reset_index(drop=True)
#plans.shape

# Encontrar duplicados en 'clean_text'
duplicated_texts = plans['clean_text'][plans['clean_text'].duplicated(keep=False)]

# Mostrar los documentos duplicados
# print(duplicated_texts)

# plans.loc[303,'clean_text']
plans.at[303,'clean_text'] = float("NaN")
# plans.loc[303,'clean_text']
plans = plans.dropna()
# Ensure all entries in 'clean_text' are strings and handle missing values
data['clean_text'] = data['clean_text'].fillna('').astype(str)
plans.shape

(309, 10)

### Preprocessing

In [32]:
#SIN BIGRAMS y sin stop words
# # Load the Spanish NLP model
# nlp = spacy.load('es_core_news_sm')

# # Preprocess documents: tokenize, remove stopwords and punctuation, and lemmatize
# def preprocess(doc):
#     spacy_doc = nlp(doc.lower())
#     return [token.lemma_ for token in spacy_doc if not token.is_stop and not token.is_punct]

# processed_docs = data['clean_text'].apply(preprocess)



In [106]:
# CON BIGRAMS
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))  # Using Spanish stopwords
# Lista de palabras personalizadas para agregar
custom_stop_words = [
    'ana imelsa guzman espinoza','junnier a salazar tobal','claudia quintanilla tartaglione','areas',
    'accion',
    'alajuelense', 'alcalde', 'alcalde', 'alcaldesa', 'alcaldia', 'alcaldia', 'ambito', 'anexo', 'anterior', 'apoyaremos', 'aqui',
    'areas',
    'brenes', 'busca', 'busca', 
    'cabo', 'cabo', 'canaza', 'candidata','canton','cantonal', 'carara', 'caso', 'cion', 'cl', 'continuar', 'continuar', 'coordinaremos', 'cosas', 'crearemos', 
    'cuales', 'cuanto', 'cuanto', 'cuatro', 'comunidad','comunidades',
    'debemos', 'debemos', 
    'elena', 'enrique', 'etc', 'etc', 'evaluacion',
    'fomentaremos', 'formato', 
    'gestionaremos', 'golfito', 'guayabo', 
    'hagamoslo', 'haremos', 
    'i','ii', 'iii', 'implementaremos', 'incluyendo', 'indicador',
    'juntos', 
    'kathia arroyo', 
    'linea', 'lineas', 'llevar', 
    'maria', 'meta', 'manera','municipalidad','mejorar',
    'ndeg', 'numero',
    'objetivo', 
    'pagina', 'partir', 'periodo', 'permitira', 'pilar', 'plan de gobierno', 'poas', 'poasena', 'porcentaje', 'posible', 'posteriormente', 
    'proyecto',
    'primer', 'primera', 'promoveremos', 'proponemos', 'provincia', 'programa de gobierno','porque liberia debe avanzar','programas',
    'responde', 'rodriguez', 
    'sa', 'samara', 'san', 'seccion', 'sino','solo',
    'tabarcia', 'tal', 'tales', 'trabajaremos', 'tres', 'tribunal interno de elecciones',
    'vamos', 'vamos', 'ver',
    "ejecutarlo", "nombre", "finalidad", "complementado", "instrumento", "debera", "fondos", "asegurar",
    "verdes", "clave", "practicas", "potenciar", "asegurar", "habilidades", "ejes", "especial", "pueden", "nuevo", "asegurar",
    "metas", "tipo", "permanente", "falta" ,"requiere", "atenienses", "rapidos", "ateniense", "empiecen", "impulso", "conlleva", "validar",
    "colectivamente", "presta", "comprende", "orden", "anchas", "intereses" , "poner", "mencionamos" , "potenciar", "vote papeletas", "limonense",
    "creara", "rio_blanco", "limonenses", "resultados", "resumen", "iniciativa", "asegurando", "clave", "asegurar", "fecha", "descritos",
    "indicadores", "estrategico", "avanza", "continuaremos", "asimismo", "santa", "dia", "gracias", "realizadas", "aprovechamiento", "dad", "nes",
    "impulsaremos", "buscaremos", "des", "municipes", "tos", "desarrollaremos", "pavones", "elecciones", "esparcimiento", "realizacion", "total",
    "nuevo", "falta", "debido", "impulsaremos", "rutinario", "estrategicas", "menciona", "pag", "escazucenos", "rafael", "tes", "aportado",
    "sirviendo", "pusc", "propone", "merecemos", "construyendo", "tienden", "siempre", "ameriten", "requeriran", "treinta", "beneficiado", "cuento",
    "bicentenario"]

# Agregar las palabras personalizadas al conjunto de stopwords
stop_words.update(custom_stop_words)


# Definir la función para limpiar el texto
def limpiar_texto(texto):
    # Eliminar palabras de una sola letra que no sean 'a' o 'I' (ajustar según tu criterio)
    texto_limpio = re.sub(r'\b(?<!\b[aAiI]\b)[a-zA-Z]\b', '', texto)
    return texto_limpio

# Aplicar la función a cada elemento de la columna de texto
cleaner_texts = plans['clean_text'].apply(limpiar_texto)

# Tokenize documents
def preprocess(text):
    # Convert to lower case and remove accents
    text = text.lower()
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    tokens = [unidecode.unidecode(word) for word in tokens]
    return tokens

tokenized_docs = [preprocess(doc) for doc in cleaner_texts]

# Build bigrams
bigram = Phrases(tokenized_docs, min_count=1, threshold=2) # Play with these parameters according to your specific needs
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Apply the trained bigram model to each document
docs_with_bigrams = [bigram_mod[doc] for doc in tokenized_docs]

# Create a dictionary representation of the documents
dictionary = Dictionary(docs_with_bigrams)

# Create a corpus from the dictionary and tokenized documents
corpus = [dictionary.doc2bow(doc) for doc in docs_with_bigrams]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\carme\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Predefined topics

In [48]:
# Define seed words for topics, should be tops bigrams
seed_words = {
    'Seguridad': ['seguridad', 'homicidio', 'asaltos','policia','hurto','hurtos','robo','robos','tacha'],
    'Capital Social': ['abstencionismo', 'participacion elecciones'],
    'Transformacion Digital': ['digital','gobernanza datos','municipalidad electronica', 'e municipalidad', ], #muni electronica y e muni son de los tramites, pero son digital, segun yo no deberian ir aca?
    'Tramites Simples': ['plan de mejora regulatoria','permiso de construccion','permisos de construccion','uso de suelos'],
    'Transparencia Municipal': ['transparencia','denuncia','acceso informacion','rendicion cuentas','participacion ciudadana','datos abiertos'],
    'Administracion Presupuestaria': ['presupuesto','ejecucion recursos','morosidad','dependencia financiera'],
    #'Compromiso con la sostenibilidad': ['residuos recolectados','basura','residuos','reciclaje','valoracion residuos','medio ambiente','sostenibilidad'], #estas dos ultimas creo que deberia estar en otra categoria y deberia ser solo de recoleccion de residuos
    #lo voy a dividir en dos:
    'Recoleccion de Residuos':['residuos','reciclaje'],
    'Medio Ambiente':['medio ambiente','sostenibilidad'],
    'Infraestructura de Transporte': ['red vial','vias','obras','infraestructura transporte','km','kilometro'],
    'Conectividad Vial': ['conectividad vial','traslado','earopuerto','puerto','presas','atraso carretera'],
    'Acceso a Servicios Publicos': ['agua','potable','tuberia','acueducto','electricidad','eliminacion de excremento'], #no inclui acceso a camion recolector de basura
    'Servicios Publicos Municipales': ['inversion servicios','servicios municipales','servicios publicos','aseo vias','aseo sitios'], #no estoy incluyendo inversion en cultura, deporte y educacion
    'Redes de Datos (internet)': ['datos moviles','velocidad descarga','5g','4g'], #no inclui 3g y 4g 
    'Redes de Voz Movil (telefonia)': ['telefonia','telefonica','llamadas telefonicas','2g'], #no inclui 2g y 3g
    'Redes de Datos Fijas (internet)': ['banda ancha','operadoras internet'],
    'Acceso a TICs en Hogares': ['acceso computadoras','acceso internet'],
    'Acceso a TICs en el Sistema Educativo': ['estudiantes computadora','pronie'],
    'Salud': ['esperanza vida','mortalidad infantil','fertilidad adolescentes'],
    'Escolaridad Media de Fuerza Laboral Actual': ['escolaridad','secundaria concluida'],
    'Habilidades Tecnicas de Fuerza Laboral Actual': ['tecnicos graduados'],
    #habilidades y competencias podria ser uno solo
    'Habilidades en Ciencia y Tecnologia de Fuerza Laboral Actual': ['ciencia tecnologia','ingenieria tecnologia','ciencias medicas','ciencias exactas','ciencias naturales','ciencias agricolas'],
    'Cobertura Educativa': ['alumnos','educacion primaria','educacion secundaria'],
    'Curriculo Completo': ['ingles','curriculo completo'], #deje afuera informatica
    'Competencias Basicas': ['matematica','espanol','evaluacion estandarizada','segundo idioma'], #deje afuera cientifica y secundaria
    'Encadenamientos Productivos': ['encadenamientos productivos','relacion venta','relacion compra'],
    'Produccion': ['pib capita','pib per capita','actividades economicas','actividad economica','emprendimiento'], #agregue emprendimiento
    'Exportacion de Bienes': ['exportaciones','productos primarios'],
    'Sector Constructivo': ['construcciones','construccion'],
    'Sector Electrico': ['interrupciones electricas','electricidad','electrico'],
    'Sector Laboral': ['desempleo','empleo','asalariado','asalariada','asegurado independiente'],
    'Genero':['brecha','genero','mujeres','mujer','desigualdad'], #revisar desigualdad
    'Cultura': ['cultura','recreacion','arte'],
    'Deporte':['deporte'],
    'Turismo': ['turismo','turistas']
}

In [ ]:
# # Boost seed words by repeating them in the documents (optional step)
# boosted_documents = []
# for doc in processed_docs:
#     boosted_doc = doc[:]
#     for topic, seeds in seed_words.items():
#         for seed in seeds:
#             if seed in doc:
#                 boosted_doc.extend([seed] * 10)  # Repeat seed words to increase their frequency
#     boosted_documents.append(boosted_doc)

# Create a dictionary and corpus
# dictionary = Dictionary(boosted_documents)
# corpus = [dictionary.doc2bow(doc) for doc in boosted_documents]

In [53]:
# Train LDA
lda = LdaModel(corpus=corpus, 
               id2word=dictionary, 
               num_topics=len(seed_words), 
               passes=15, random_state=0)

In [54]:
# Display topics
topics = lda.print_topics()
for topic_id, topic in topics:
    print(f"Topic {topic_id}: {topic}")

Topic 22: 0.006*"turrialba" + 0.004*"turrialba." + 0.004*"progresista_programa" + 0.004*"municipalidad_turrubares" + 0.004*"2024-_2028" + 0.004*"gobierno_|" + 0.003*"solo" + 0.003*"turrialba," + 0.002*"3-" + 0.002*"comunidad"
Topic 13: 0.009*"montes_oca" + 0.004*"medible_(measurable):" + 0.003*"relevante_(relevant):" + 0.003*"exito_proyecto" + 0.003*"gobierno_2024" + 0.003*"temporalmente_definido" + 0.003*"|_kathia" + 0.003*"arroyo_alcaldesa" + 0.003*"montes_oca." + 0.003*"plpgarabito@gmail.com"
Topic 16: 0.002*"canton" + 0.001*"instituciones_afines," + 0.001*"acompanamiento_sector" + 0.001*"respaldadas_parte" + 0.001*"local" + 0.001*"." + 0.001*"atraccion_inversion" + 0.001*"desarrollo_empresarial" + 0.001*"empresas_vean" + 0.001*"*"
Topic 23: 0.008*"podemos_tres" + 0.008*"vote_unidos" + 0.008*"podemos_2024-2028" + 0.008*"papeletas_plan" + 0.008*"renacer_unidos" + 0.001*"limon" + 0.001*"canton_limon," + 0.001*"plan_renacer," + 0.001*"cuatro_distritos," + 0.001*"renacer,"
Topic 18: 0.0

## Usando uno solo

In [107]:
# CON BIGRAMS
tokenized_doc = [preprocess(doc) for doc in cleaner_texts[291:292]]

# Build bigrams
bigram = Phrases(tokenized_doc, min_count=1, threshold=2) # Play with these parameters according to your specific needs
bigram_mod = gensim.models.phrases.Phraser(bigram)

# Apply the trained bigram model to each document
docs_with_bigrams = [bigram_mod[doc] for doc in tokenized_doc]

# Create a dictionary representation of the documents
dictionary = Dictionary(docs_with_bigrams)

# Create a corpus from the dictionary and tokenized documents
corpus = [dictionary.doc2bow(doc) for doc in docs_with_bigrams]

In [108]:
cleaner_texts[291]

'Sección  ____________________________________________________________________ 1\r\n  Definición de Ejes  Variaciones __________________________________________________ 1\r\n  Acrónimo _______________________________________________________________________ 1\r\nSección II – Desglose de Objetivos específicos para la Municipalidad ____________________ 5\r\nObjetivos Generales Específicos para la Municipalidad _______________________________ 7\r\n    Modelo de trabajo para proyectos/tareas basado en principios  modelos de PM  Agile Scrum. ___ 7\r\n    Desarrollar un sistema de evaluación del desempeño para los programas. ______________________ 8\r\n         1. Recopilación de datos: ___________________________________________________________ 8\r\n         2. Definición de indicadores clave de rendimiento (KPIs): _______________________________ 8\r\n         3. Análisis de datos: ________________________________________________________________ 8\r\n    Plan de Implementación de Certificaci

In [109]:
# Train LDA
lda = LdaModel(corpus=corpus, 
               id2word=dictionary, 
               num_topics=5, 
               passes=15, random_state=0)

In [110]:
# Display topics
topics = lda.print_topics()
for topic_id, topic in topics:
    print(f"Topic {topic_id}: {topic}")

Topic 0: 0.000*"." + 0.000*"ii." + 0.000*"areas" + 0.000*"61" + 0.000*"proyectos" + 0.000*"objetivos" + 0.000*"implementacion" + 0.000*"si" + 0.000*"empleados" + 0.000*"seguridad"
Topic 1: 0.000*"." + 0.000*"seguridad" + 0.000*"areas" + 0.000*"recursos" + 0.000*"ii." + 0.000*"iii." + 0.000*"proyectos" + 0.000*"61" + 0.000*"1-)" + 0.000*"puedan"
Topic 2: 0.000*"." + 0.000*"ii." + 0.000*"areas" + 0.000*"proyectos" + 0.000*"seguridad" + 0.000*"recursos" + 0.000*"trabajo" + 0.000*"implementacion" + 0.000*"iii." + 0.000*"61"
Topic 3: 0.000*"." + 0.000*"areas" + 0.000*"ii." + 0.000*"proyectos" + 0.000*"recursos" + 0.000*"61" + 0.000*"cada" + 0.000*"oportunidades" + 0.000*"evaluacion" + 0.000*"iii."
Topic 4: 0.008*"." + 0.003*"ii." + 0.003*"areas" + 0.003*"proyectos" + 0.003*"recursos" + 0.002*"seguridad" + 0.002*"61" + 0.002*"iii." + 0.002*"empleados" + 0.002*"dentro"
